In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import confusion_matrix

# Load the dataset
data = pd.read_csv('./data/augmented_data.csv')

# Separate the features and the target variable
X = data.drop('success', axis=1)
y = data['success']

In [2]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the number of folds for cross-validation
k = 5

In [3]:
# Calculate the correlation between features and the target variable
correlations = X_train.corrwith(y_train)

# Set a threshold for considering high correlation
threshold = 0.1

# Find the highly correlated features
highly_correlated_features = correlations[correlations.abs() >= threshold]

# Print the highly correlated features
print("Highly Correlated Features:")
print(highly_correlated_features)

Highly Correlated Features:
shs_gpa                                                              0.188895
p2                                                                   0.203887
p5                                                                   0.149366
p14                                                                  0.130378
p21                                                                  0.129414
p23                                                                  0.156769
p25                                                                  0.123879
p26                                                                  0.113198
p30                                                                  0.164153
p31                                                                  0.105196
p33                                                                  0.145390
s9                                                                   0.150483
s18                                 

### Classical Model

In [4]:
# Create a k-fold cross-validation object
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear')
svm_cv_scores = cross_val_score(svm_model, X_train, y_train, cv=kf)  # Perform k-fold cross-validation
svm_accuracy = svm_cv_scores.mean()

# Decision Tree (DT) model
dt_model = DecisionTreeClassifier()
dt_cv_scores = cross_val_score(dt_model, X_train, y_train, cv=kf)  # Perform k-fold cross-validation
dt_accuracy = dt_cv_scores.mean()

# Neural Network (NN) model
nn_model = MLPClassifier()
nn_cv_scores = cross_val_score(nn_model, X_train, y_train, cv=kf)  # Perform k-fold cross-validation
nn_accuracy = nn_cv_scores.mean()

# Train the models on the full training set
svm_model.fit(X_train, y_train)
dt_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train)

# Make predictions on the test set
svm_predictions = svm_model.predict(X_test)
dt_predictions = dt_model.predict(X_test)
nn_predictions = nn_model.predict(X_test)

C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_

In [5]:
# Calculate evaluation metrics
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
svm_auc = roc_auc_score(y_test, svm_predictions)

dt_report = classification_report(y_test, dt_predictions, output_dict=True)
dt_auc = roc_auc_score(y_test, dt_predictions)

nn_report = classification_report(y_test, nn_predictions, output_dict=True)
nn_auc = roc_auc_score(y_test, nn_predictions)

# Calculate confusion matrix for each model
svm_confusion = confusion_matrix(y_test, svm_predictions)
dt_confusion = confusion_matrix(y_test, dt_predictions)
nn_confusion = confusion_matrix(y_test, nn_predictions)

# Create a DataFrame for the results
results_df = pd.DataFrame({
    'Model': ['SVM', 'Decision Tree', 'Neural Network'],
    'CV Accuracy': [svm_accuracy, dt_accuracy, nn_accuracy],
    'Accuracy': [svm_model.score(X_test, y_test), dt_model.score(X_test, y_test), nn_model.score(X_test, y_test)],
    'F1-Score': [svm_report['macro avg']['f1-score'], dt_report['macro avg']['f1-score'], nn_report['macro avg']['f1-score']],
    'Precision': [svm_report['macro avg']['precision'], dt_report['macro avg']['precision'], nn_report['macro avg']['precision']],
    'Recall': [svm_report['macro avg']['recall'], dt_report['macro avg']['recall'], nn_report['macro avg']['recall']],
    'AUC': [svm_auc, dt_auc, nn_auc],
    'Confusion Matrix': [svm_confusion, dt_confusion, nn_confusion]
})

# Print the results
print("Evaluation Metrics:")
print(results_df)

Evaluation Metrics:
            Model  CV Accuracy  Accuracy  F1-Score  Precision    Recall  \
0             SVM     0.742385  0.818182  0.817717   0.818182  0.821462   
1   Decision Tree     0.766380  0.813131  0.812744   0.813673  0.816875   
2  Neural Network     0.886378  0.914141  0.913646   0.912643  0.915833   

        AUC      Confusion Matrix  
0  0.821462  [[86, 23], [13, 76]]  
1  0.816875  [[85, 24], [13, 76]]  
2  0.915833   [[98, 11], [6, 83]]  


In [6]:
# Obtain feature importances for linear SVM model
svm_feature_importances = abs(svm_model.coef_[0])

# Create a DataFrame for feature importances
importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'SVM Importance': svm_feature_importances
})

# Sort the DataFrame by SVM Importance values in descending order
importances_df = importances_df.sort_values(by='SVM Importance', ascending=False)

# Display the top 10 feature importances
top_10_importances = importances_df.head(10)
print("Top 10 Feature Importances (SVM Linear):")
print(top_10_importances)

Top 10 Feature Importances (SVM Linear):
                                               Feature  SVM Importance
92   classroom_org_I forgot already, might be just ...        1.676340
166                                       s25_Leave it        1.427534
72                   program_BS Information Technology        1.397573
151                         mothers_employment_Retired        1.340495
108                         school_org_Project Manager        1.312652
157                         income_P50,000 to P100,000        1.267727
102                                 school_org_Auditor        1.239745
145      mothers_employment_Disabled, not able to work        1.233484
139  mothers_education_Not Applicable (no work, not...        1.231842
174                          s26_More than 4 hours/day        1.177111


In [7]:
# Create a DataFrame for feature importances
importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': dt_model.feature_importances_
})

# Sort the DataFrame by importance values in descending order
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Display the top 10 feature importances
top_10_importances = importances_df.head(10)
print("DT Top 10 Feature Importances:")
print(top_10_importances)

DT Top 10 Feature Importances:
     Feature  Importance
65       s19    0.072260
6    shs_gpa    0.057699
1    english    0.047243
2    reading    0.041109
5   abstract    0.040922
3    science    0.038781
7         p1    0.035635
61       s15    0.035396
43       p37    0.032524
29       p23    0.030959


In [8]:
# Obtain feature importances for Neural Network model
nn_feature_importances = abs(nn_model.coefs_[0].mean(axis=0))

# Create a DataFrame for feature importances
importances_df = pd.DataFrame({
    'Feature': X_train.columns[:len(nn_feature_importances)],
    'NN Importance': nn_feature_importances
})

# Sort the DataFrame by NN Importance values in descending order
importances_df = importances_df.sort_values(by='NN Importance', ascending=False)

# Display the top 10 feature importances
top_10_importances = importances_df.head(10)
print("Top 10 Feature Importances (Neural Network):")
print(top_10_importances)

Top 10 Feature Importances (Neural Network):
                    Feature  NN Importance
13                       p7       0.028392
33                      p27       0.026887
11                       p5       0.023179
69                      s23       0.017305
97  classroom_org_President       0.016480
56                      s10       0.014477
80            strand_TVL-HE       0.014214
34                      p28       0.013899
49                       s3       0.013504
66                      s20       0.013022


### Ensemble Model

In [9]:
# Create the ensemble model using Voting Classifier
ensemble_model = VotingClassifier(
    estimators=[('svm', svm_model), ('dt', dt_model), ('nn', nn_model)],
    voting='hard'
)

# Perform k-fold cross-validation on the ensemble model
ensemble_cv_scores = cross_val_score(ensemble_model, X_train, y_train, cv=kf)
ensemble_accuracy = ensemble_cv_scores.mean()

# Train the ensemble model on the full training set
ensemble_model.fit(X_train, y_train)

# Make predictions on the test set
ensemble_predictions = ensemble_model.predict(X_test)

# Calculate evaluation metrics for the ensemble model
ensemble_report = classification_report(y_test, ensemble_predictions, output_dict=True)
ensemble_auc = roc_auc_score(y_test, ensemble_predictions)

C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_

In [10]:
# Create the ensemble model using Bagging Classifier
bagging_model = BaggingClassifier(base_estimator=SVC(kernel='linear'), n_estimators=10)
bagging_cv_scores = cross_val_score(bagging_model, X_train, y_train, cv=kf)  # Perform k-fold cross-validation
bagging_accuracy = bagging_cv_scores.mean()

# Train the model on the full training set
bagging_model.fit(X_train, y_train)

# Make predictions on the test set
bagging_predictions = bagging_model.predict(X_test)

# Calculate evaluation metrics
bagging_report = classification_report(y_test, bagging_predictions, output_dict=True)
bagging_auc = roc_auc_score(y_test, bagging_predictions)

In [11]:
# Create the ensemble model using Stacking Classifier
stacking_model = StackingClassifier(
    estimators=[('SVM', svm_model), ('Decision Tree', dt_model), ('Neural Network', nn_model)],
    final_estimator=SVC(kernel='linear')
)

stacking_cv_scores = cross_val_score(stacking_model, X_train, y_train, cv=kf)  # Perform k-fold cross-validation
stacking_accuracy = stacking_cv_scores.mean()

# Train the model on the full training set
stacking_model.fit(X_train, y_train)

# Make predictions on the test set
stacking_predictions = stacking_model.predict(X_test)

# Calculate evaluation metrics
stacking_report = classification_report(y_test, stacking_predictions, output_dict=True)
stacking_auc = roc_auc_score(y_test, stacking_predictions)

C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Jaydee\anaconda3\lib\site-packages\sklearn\neural_network\_

In [12]:
# Calculate confusion matrix for each model
ensemble_confusion = confusion_matrix(y_test, ensemble_predictions)
bagging_confusion = confusion_matrix(y_test, bagging_predictions)
stacking_confusion = confusion_matrix(y_test, stacking_predictions)

# Create a DataFrame for the ensemble models
ensemble_results = pd.DataFrame({
    'Model': ['Ensemble', 'Bagging', 'Stacking'],
    'CV Accuracy': [ensemble_accuracy, bagging_accuracy, stacking_accuracy],
    'Accuracy': [ensemble_model.score(X_test, y_test), bagging_model.score(X_test, y_test), stacking_model.score(X_test, y_test)],
    'F1-Score': [ensemble_report['macro avg']['f1-score'], bagging_report['macro avg']['f1-score'], stacking_report['macro avg']['f1-score']],
    'Precision': [ensemble_report['macro avg']['precision'], bagging_report['macro avg']['precision'], stacking_report['macro avg']['precision']],
    'Recall': [ensemble_report['macro avg']['recall'], bagging_report['macro avg']['recall'], stacking_report['macro avg']['recall']],
    'AUC': [ensemble_auc, bagging_auc, stacking_auc],
    'Confusion Matrix': [ensemble_confusion, bagging_confusion, stacking_confusion]
})

# Print the results of the ensemble models
print("Ensemble Model Results:")
print(ensemble_results)

Ensemble Model Results:
      Model  CV Accuracy  Accuracy  F1-Score  Precision    Recall       AUC  \
0  Ensemble     0.866189  0.934343  0.933964   0.932897  0.936244  0.936244   
1   Bagging     0.787891  0.858586  0.857407   0.856835  0.858159  0.858159   
2  Stacking     0.882533  0.939394  0.938636   0.939733  0.937738  0.937738   

       Confusion Matrix  
0   [[100, 9], [4, 85]]  
1  [[94, 15], [13, 76]]  
2   [[104, 5], [7, 82]]  


### Batch Prediction

In [13]:
# Load the test dataset
new_data = pd.read_csv('./data/testing.csv')

# Separate the features
X_new = new_data.drop('success', axis=1)
y_true = new_data['success']

# Predict using the SVM model
svm_predictions = svm_model.predict(X_new)

# Predict using the Decision Tree model
dt_predictions = dt_model.predict(X_new)

# Predict using the Neural Network model
nn_predictions = nn_model.predict(X_new)

# Predict using the Ensemble(Voting)
ensemble_predictions = ensemble_model.predict(X_new)

# Predict using the Ensemble(Bagging)
bagging_predictions = bagging_model.predict(X_new)

# Predict using the Ensemble(Stacking)
stacking_predictions = stacking_model.predict(X_new)

# Create a DataFrame for the predictions
predictions_df = pd.DataFrame({'SVM Predictions': svm_predictions,
                               'DT Predictions': dt_predictions,
                               'NN Predictions': nn_predictions,
                               'Voting Predictions': ensemble_predictions,
                               'Bagging Predictions': bagging_predictions,
                               'Stacking Predictions': stacking_predictions,
                               'Actual': y_true})

# Add a column to indicate incorrect predictions
predictions_df['SVM Correct'] = predictions_df['SVM Predictions'] == predictions_df['Actual']
predictions_df['DT Correct'] = predictions_df['DT Predictions'] == predictions_df['Actual']
predictions_df['NN Correct'] = predictions_df['NN Predictions'] == predictions_df['Actual']
predictions_df['Voting Correct'] = predictions_df['Voting Predictions'] == predictions_df['Actual']
predictions_df['Bagging Correct'] = predictions_df['Bagging Predictions'] == predictions_df['Actual']
predictions_df['Stacking Correct'] = predictions_df['Stacking Predictions'] == predictions_df['Actual']

# Calculate accuracy for each model
svm_accuracy = predictions_df['SVM Correct'].mean()
dt_accuracy = predictions_df['DT Correct'].mean()
nn_accuracy = predictions_df['NN Correct'].mean()
ensemble_accuracy = predictions_df['Voting Correct'].mean()
bagging_accuracy = predictions_df['Bagging Correct'].mean()
stacking_accuracy = predictions_df['Stacking Correct'].mean()

# Print the comparison table and accuracy
print("Comparison Table:")
print(predictions_df)
print("\nAccuracy:")
print("SVM Accuracy:", svm_accuracy)
print("DT Accuracy:", dt_accuracy)
print("NN Accuracy:", nn_accuracy)
print("Voting Accuracy:", ensemble_accuracy)
print("Bagging Accuracy:", bagging_accuracy)
print("Stacking Accuracy:", stacking_accuracy)

Comparison Table:
   SVM Predictions  DT Predictions  NN Predictions  Voting Predictions  \
0                1               1               1                   1   
1                0               0               0                   0   
2                0               0               0                   0   
3                0               1               1                   1   
4                0               0               0                   0   
5                1               0               1                   1   
6                0               0               0                   0   
7                1               1               1                   1   
8                1               1               0                   1   
9                1               1               1                   1   

   Bagging Predictions  Stacking Predictions  Actual  SVM Correct  DT Correct  \
0                    1                     1       1         True        True   
1    